In [2]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

In [1]:
HOPSWORKS_PROJECT_NAME = "taxi_demand_pravachan"

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / ".env")  # load the .env environment variable file by passing in its path
HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]  # extract the api-key-var from the file



hUNGlgi1x2D9HeR5.sVRMiz9rNNCjW4G0jKMv8wN74oV2n9kQ55mmp86PrONjJTm5cR8LvYwLY7NodXkt


In [4]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data  # func

from_year = 2022  # starting year
to_year = datetime.now().year   # ending year which is now
print(f"Downloading raw data from {from_year} to {to_year}")
rides = pd.DataFrame()  # initlize empty dataframe to store all rides in date range
# iterate every year from    starting-year to inclusive ending year
for year in range(from_year, to_year+1):
    # calling load-raw-data-func passing in cur-year, downloads raw-data of all months of cur-year from website and validates it and returns that data-frame
    rides_one_year = load_raw_data(year)
    # add the rides-data-frame for cur year to the all-rides-df
    rides = pd.concat([rides, rides_one_year])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
File 2022-01 is already in local storage
File 2022-02 is already in local storage
File 2022-03 is already in local storage
File 2022-04 is already in local storage
File 2022-05 is already in local storage
File 2022-06 is already in local storage
File 2022-07 is already in local storage
File 2022-08 is already in local storage
File 2022-09 is already in local storage
File 2022-10 is already in local storage
File 2022-11 is already in local storage
File 2022-12 is already in local storage
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2024-05 file is not available
2024-06 file is not available
2024-07 file is not available
2024-08 file is not available
2024-09 file is not available
2024-10 file is not available
2024-11 file is not available
2024-12 file is not available


In [5]:
print(f"{len(rides)}")   # number of rows of data, each row represents a ride

91033132


In [6]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)  # transforms rides-df-raw into time-series data-frame is returned

100%|██████████| 263/263 [00:05<00:00, 52.28it/s]


In [15]:
import hopsworks  

In [16]:
# creating hopsworks-project by passing project name and api-key
project = hopsworks.login(project=HOPSWORKS_PROJECT_NAME, api_key_value=HOPSWORKS_API_KEY)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


In [17]:
# hopsworks-project.get-feature-store() gest pointer to the feature store
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [18]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION =  3  # name and version of feature group

In [21]:
# creating feature-group-obj feature-store-obj.create()
# primary-key: unique identifier for each row in data
# event-time: is the column that stores the tiemstamp behind those events
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

In [22]:
# save data to feature-group
feature_group.insert(ts_data, write_options={"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/835736/fs/831559/fg/945531


Uploading Dataframe: 0.00% |          | Rows 0/5371512 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x12fcfff90>, None)